<a href="https://colab.research.google.com/github/rby011/hf_study/blob/main/01_train_basic_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎬 영화 리뷰 데이터로 BERT 모델 감성 분석하도록 훈련 시키기

### 📂 Load dataset

- huggingface `datasets` package
  ```
    pip install datasets
  ```

- nsmc dataset
  - https://huggingface.co/datasets/nsmc

In [30]:
!pip install datasets transformers[torch] torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

nsmc_dataset = load_dataset("nsmc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
nsmc_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [4]:
#####################################################################################
# DataSetDict 형태로 데이터 구조 , 내용 살펴보기
#####################################################################################

# document 는 리뷰 , label 은 리뷰의 긍정/부정 (positive / negative)
display(nsmc_dataset['train'].features)

# label 의 명칭은 'negative' , 'positive' 인데 여기서는 0, 1 로 표현되어 있음
display(nsmc_dataset['train'][0])

# label 명에 따른 label id 는 `label` feature 를 str2int 로 확인
display(nsmc_dataset['train'].features['label'].str2int('positive'),
        nsmc_dataset['train'].features['label'].str2int('negative'))

{'id': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

1

0

In [5]:
#####################################################################################
# 판다스 데이터 프레임으로 변환해서 좀더 세밀히 살펴보기
#####################################################################################

# pandas 데이터 프레임으로 변환해서 자세히 살펴보기
nsmc_df = nsmc_dataset['train'].to_pandas()
display(nsmc_df.head())

# (⚠️ 주의)분류 문제인 경우 label 불균등을 확인해보는 것이 중요
# - 만일 label 이 매우 불균등한 상태라면 over sampling, under sampling 등 적용 필요
# - negative 50.1% , positive 49.9%
display(nsmc_df.groupby('label').apply(lambda x:len(x)/len(nsmc_df)))

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


label
0    0.501153
1    0.498847
dtype: float64

In [6]:
# 리뷰 문자열 길이 알아보기
nsmc_df['review_length'] = nsmc_df['document'].str.len()
nsmc_df['review_length'].describe()

count    150000.000000
mean         35.203353
std          29.532097
min           0.000000
25%          16.000000
50%          27.000000
75%          42.000000
max         146.000000
Name: review_length, dtype: float64

#### 📂 preprocess

자연어 문제의 기본은 `Tokenization` 으로 매우 중요. 예를 들어, `Tokenization` 의 방식에 따라 의미가 달라짐

여기서는 BERT Multilangual Tokenizer 를 그대로 사용할 것 (BERT 모델에 들어 있는 것)

`Auto` 붙은 것은 pretrained 모델을 갖고 올 때 그에 맞는 class 를 retrieval 해주는 것일 뿐

모델은 https://huggingface.co/google-bert/bert-base-multilingual-cased  이 것 가지고 올 것임

※ 한국어 Tokenizer , KoBERT 등 사용하는 것도 괜찮음


In [7]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tok


BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
#####################################################################################
# 문장 tokenize 해보기
# - 한국어만으로 학습된 tokenizer 가 아닌 다국어로 학습된 것임에도 어느정도는 잘 함
# - `##` 이 들어가지 않은 것은 단어가 시작되는 부분, 들어간 것은 이전 단어와 이어지는 것
#####################################################################################
seq = "청춘 영화의 최고봉."

tok.tokenize(seq)

['청', '##춘', '영화', '##의', '최고', '##봉', '.']

In [9]:
# input_ids 는 토큰을 vocabulary 에 포함된 토큰이 id 로 매핑한 결과
# attention_mask 는 해당 토큰이 실제 데이터인지 여부를 마킹 (패팅 토큰 등은 0)
# token_type_ids 는 문장을 구분하는데 사용됨.
# - 예를 들어 premise, hypothesis 의 NLI (함의/모순/중립 판단) 하는 경우 두 개의 문장을 서로 구분하는 역할
from pprint import pprint
pprint(tok(seq))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 9751, 97707, 42428, 10459, 83491, 118989, 119, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [10]:
# 패딩으로 채워진 두 번째 짧은 문장에서 패팅에 해당 하는 위치는 attention mask 가 0 임
seq2 = '청춘'
pprint(tok([seq, seq2], padding=True))

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0]],
 'input_ids': [[101, 9751, 97707, 42428, 10459, 83491, 118989, 119, 102],
               [101, 9751, 97707, 102, 0, 0, 0, 0, 0]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]}


### 참고

NLI Task 의 경우 아래 같은 코드를 통해 입력을 모델에게 줌 (예시임. GPT)

In [11]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

premise = "The quick brown fox jumps over the lazy dog. It did not stop running."
hypothesis = "A fast animal is moving."

input_text = "[CLS] " + premise + " [SEP] " + hypothesis + " [SEP]"
tokenized_output = tokenizer.tokenize(input_text)

input_ids = tokenizer.convert_tokens_to_ids(tokenized_output)

# 🔑 [CLS] , [SEP] 이 포함된 모든 토큰에 대해 `1` 로 설정
attention_mask = [1] * len(input_ids)

# 🔑 premise 는 0 , hypothesis 는 1 로 설정
sep_indices = [i for i, token in enumerate(tokenized_output) if token == "[SEP]"]
token_type_ids = [0 if i <= sep_indices[0] else 1 for i in range(len(tokenized_output))]


In [12]:
# 로딩한 데이터(DataSetDict)의 문장을 최대 32 길이로 하고 짧은 경우 padding 하여 tokenize
def tokenizer(data):
    return tok(data['document'], max_length=32, truncation=True, padding='max_length')

In [13]:
nsmc_dataset_toeknized = nsmc_dataset.map(tokenizer)

In [14]:
# padding 정확하게 들어간건지 확인해야 함
pprint(nsmc_dataset_toeknized['train'][0])

{'attention_mask': [1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0,
                    0],
 'document': '아 더빙.. 진짜 짜증나네요 목소리',
 'id': '9976970',
 'input_ids': [101,
               9519,
               9074,
               119005,
               119,
               119,
               9708,
               119235,
               9715,
               1192

### 🔄 load model for finetuing

`bart-base` 모델을 기반으로 각 task 별로 적절한 layer 를 덧붙이는 것이 필요함.

예를 들면, 문장 분류 작업을 위해서는 dense layer 와 softmax 를 붙여서 사용을 해야 하고,

질의 응답 같은 경우는 base 모델이 출력하는 토큰 중에서 어디서 부터 어디까지가 답변인지를 식별하는 layer 를 붙여두어야 함

transformer 패키지는 각각에 대해서 사전 정의된 layer 를 붙여서 모델을 로딩하도록 할 수 있도록 하고 있고

`BertQuestionAnswering`, `BertSequenceClassification` 등의 클래스가 이에 해당함

In [15]:
# device 정의
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
# trainig 에 필요한 설정 정의
num_train_epochs = 2
learning_rate = 2e-7
batch_size = 128

In [17]:
#
# task 에 적합하도록 layer 를 붙여서 model 로딩
#
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 💪 finetune with pytorch

- toeknizer 와 연계해서 dataloader 만들기

In [18]:
# 최대 토큰 길이 32 로 하고 padding 추가해서 토큰화
def tokenizer(data):
    return tok(data['document'], max_length=32, truncation=True, padding='max_length')

In [19]:
# 모든 데이터를 대상으로 토큰화
nsmc_dataset_toeknized = nsmc_dataset.map(tokenizer)

In [20]:
# train 데이터에 대해서 토큰화하여 dataloader 로 만들기
from torch.utils.data import DataLoader

tr_ds = nsmc_dataset_toeknized['train'].remove_columns(['id', 'document'])
tr_ds.set_format(type='torch')
tr_dl = DataLoader(tr_ds, batch_size = batch_size)

In [21]:
next(iter(tr_dl))

{'label': tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
         0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
         0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
         0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
         0, 0, 1, 1, 0, 1, 0, 1]),
 'input_ids': tensor([[  101,  9519,  9074,  ...,     0,     0,     0],
         [  101,   100,   119,  ..., 16439,   102,     0],
         [  101,   100,   102,  ...,     0,     0,     0],
         ...,
         [  101,  9358, 12508,  ...,     0,     0,     0],
         [  101,  9519, 25503,  ...,     0,     0,     0],
         [  101, 10150, 10954,  ...,  9568, 12310,   102]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 

In [22]:
# test 데이터를 토큰화해서 dataloader 만들기
val_ds = nsmc_dataset_toeknized['test'].remove_columns(['id', 'document'])
val_ds.set_format(type='torch')
val_dl = DataLoader(val_ds, batch_size = batch_size)

In [23]:
next(iter(val_dl))

{'label': tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
         0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
         1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
         0, 0, 1, 1, 0, 1, 0, 1]),
 'input_ids': tensor([[   101,   8911,    100,  ...,      0,      0,      0],
         [   101,    144,  11490,  ...,      0,      0,      0],
         [   101,   9303,  21711,  ...,      0,      0,      0],
         ...,
         [   101,  25805, 118990,  ...,  35506,  16439,    102],
         [   101,  80956,   9511,  ...,      0,      0,      0],
         [   101,   9670,  89523,  ...,    119,    119,    102]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         

#### 🏋️학습코드 살펴보기

<font color='yellow'> ★ loss 선택하기 </font>

- 이진 분류의 문제를 풀기 위해 CrossEntropy 함수를 선택하였고 그 입력으로 양성으로 예측한 확률과 양성/음성 라벨 값(1/0) 을 넣었음

- 모델 예측 결과인 `pred` 는 다음과 같은 형식임

  ```python
   SequenceClassifierOutput(
      loss=None,
      logits=tensor([[ 0.1504, -0.0221],
        [ 0.0619, -0.0455],
        [ 0.0928, -0.0023],
        [ 0.1067, -0.0332],
        [ 0.0868, -0.0205],
  ```
- 위 pred 의 logits 중 1번째 것만을 취하기 위해 t() 를 한 것

In [25]:
import numpy as np
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# Accuracy 함수 정의
def acc(pred , label):
  pred = torch.round(pred.squeeze())
  return torch.sum(pred == label.squeeze()).item()

model.to(device)

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

# for each EPOCH
for epoch in range(num_train_epochs):
  train_lossses = []
  train_acc = 0.0
  model.train()
  # for each BATCH in an epoch
  for step, batch in enumerate(tqdm(tr_dl)):
    label = batch['label'].to(device)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)

    model.zero_grad()
    pred = model(input_ids, attention_mask, token_type_ids)
    loss = criterion(torch.sigmoid(pred.logits.t()[1]), label.float())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_lossses.append(loss.item())
    train_acc = train_acc + acc(pred.logits.argmax(dim=1), label)

  # one epoch train
  train_loss = np.mean(train_lossses)
  train_acc = train_acc / len(tr_ds)
  print(f'# epoch: {epoch+1}, train loss: {train_loss:.4f}, train acc: {train_acc:.4f}')

  # eval after one epoch
  model.eval()
  val_lossses = []
  val_acc = 0.0
  # for each BATCH in an epoch
  for step, batch in enumerate(tqdm(val_dl)):
    label = batch['label'].to(device)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)

    pred = model(input_ids, attention_mask, token_type_ids)
    loss = criterion(torch.sigmoid(pred.logits.t()[1]), label.float())

    val_lossses.append(loss.item())
    val_acc =  val_acc + acc(pred.logits.argmax(dim=1), label)

  val_loss = np.mean(val_lossses)
  val_acc = val_acc / len(val_ds)

  print(f'# epoch: {epoch+1}, validation loss: {val_loss:.4f}, train acc: {val_acc:.4f}')

100%|██████████| 1172/1172 [04:09<00:00,  4.70it/s]


# epoch: 1, train loss: 307.7467, train acc: 0.6072


100%|██████████| 391/391 [00:27<00:00, 14.14it/s]


# epoch: 1, validation loss: 307.0408, train acc: 0.6990


100%|██████████| 1172/1172 [04:07<00:00,  4.73it/s]


# epoch: 2, train loss: 303.5219, train acc: 0.7133


100%|██████████| 391/391 [00:27<00:00, 13.98it/s]

# epoch: 2, validation loss: 304.6934, train acc: 0.7328


In [ ]:
# 모델 출력 살펴보기
# - SequenceClassifierOutput 클래스이고 loss, logits 속성이 있고
# - logits 속성은 tensor 형태로 양성/음성에 대한 logit 을 줌

# 모델 GPU 에 넣기
# model.to(device)

# 모델에 줄 입력 값들(첫 배치) GPU 에 넣기
batch1 = next(iter(tr_dl))
label = batch1['label'].to(device)
input_ids = batch1['input_ids'].to(device)
attention_mask = batch1['attention_mask'].to(device)
token_type_ids = batch1['token_type_ids'].to(device)

# 모델에 입력값 넣기
pred = model(input_ids, attention_mask, token_type_ids)

display(pred.logits.t())

# 첫 입력, 정답, 예측 결과 살펴보기
tok.decode(input_ids[0]) , label[0], torch.sigmoid(pred.logits.t()[1])[0]

## 💪 finetune with 허깅페이스

In [35]:
from transformers import Trainer, TrainingArguments

output_dir = './trainer_test'

training_args = TrainingArguments(output_dir=output_dir,
                                  num_train_epochs=num_train_epochs,
                                  learning_rate=learning_rate,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy='epoch',
                                  logging_steps=len(nsmc_dataset['train'])//batch_size,
                                  save_strategy='epoch',
                                  fp16=True,
                                  push_to_hub=False)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [36]:
!pip show accelerate

Name: accelerate
Version: 0.27.2
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
